In [4]:
import os
import pickle
from pathlib import Path
from typing import Generator

import numpy as np
import pandas as pd
import scikit_posthocs as sp
from IPython.display import HTML, display
from scipy import stats
from yaml import safe_load
from jmetal.lab.statistical_test.functions import (
    friedman_aligned_rank_test,
    friedman_aligned_ph_test,
)


pd.set_option("display.max_columns", 100)


def friedmann_nemenyi_test(
    unravelled_detailed_results: dict[str : dict[tuple[str, str], list[float]]],
) -> None:
    for metric in unravelled_detailed_results.keys():
        metric_df = pd.DataFrame(unravelled_detailed_results[metric])
        metric_vals = [metric_df[col].values for col in metric_df.columns]
        out = stats.friedmanchisquare(*metric_vals)
        print(
            f"Friedman Test {metric}: statistic={out.statistic:.3f}, pvalue={out.pvalue:.3f}"
        )
        metric_vals = np.array(metric_vals).T
        out = sp.posthoc_nemenyi_friedman(metric_vals)
        print("Nemenyi post-hoc test")
        feature_names = metric_df.columns
        out.index = feature_names
        out.columns = feature_names
        out = out.sort_index(axis=0).sort_index(axis=1)
        display(out)


def aligned_friedmann_holm_test(
    unravelled_detailed_results: dict[str : dict[tuple[str, str], list[float]]],
) -> None:
    for metric in unravelled_detailed_results.keys():
        metric_df = pd.DataFrame(unravelled_detailed_results[metric])
        # metric_vals = np.asarray([metric_df[col].values for col in metric_df.columns])
        metric_vals = metric_df.values

        out = friedman_aligned_rank_test(metric_vals)
        chi2_stat = out.loc['Aligned Rank stat'].iloc[0]
        p_value = out.loc['p-value'].iloc[0]
        print(f"Aligned-rank Friedman χ² {metric} = {chi2_stat:.3f}, p = {p_value:.3f}")

        z_vals, out, _ = friedman_aligned_ph_test(
            metric_vals, apv_procedure="Holm"  # Holm step-down correction
        )
        print("Holm post-hoc test")
        feature_names = metric_df.columns
        out.index = feature_names
        out.columns = feature_names
        out = out.sort_index(axis=0).sort_index(axis=1)
        display(out)


def present_results(
    paths: Generator,
    val_method: str = "lopo",
    remove_xgboost: bool = False,
    remove_chronos_small_from_test: bool = False,
    which_test: str = "friedmann-nemenyi",
    test_args: dict = {},
) -> None:
    results = []
    for reports_path in paths:
        report = pd.read_csv(reports_path, index_col=0)
        conf = safe_load(open(reports_path.parent / ".hydra/config.yaml"))
        if conf["validation_method"]["_target_"].split(".")[-1].lower() != val_method:
            continue
        model_name: str = conf["model"]["model"]["_target_"].split(".")[-1]
        features_name = (
            conf["feature_extractor"]["_target_"].split(".")[-1]
            if "model_name" not in conf["feature_extractor"]
            else conf["feature_extractor"]["model_name"]
        )
        validation_method = conf["validation_method"]["_target_"].split(".")[-1]
        if "aggregator" not in conf:
            aggregator = "MeanTimeAggregator"
        else:
            aggregator = (
                conf["aggregator"]["_target_"].split(".")[-1]
                if "_target_" in conf["aggregator"]
                else None
            )
        report_results = {}
        for col in report.columns:
            report_results[f"{col} avg"] = report[col].mean()
            report_results[f"{col} sem"] = report[col].sem() * 1.98  # 95% CI

        dataset = conf["dataset"]
        side = conf["side"]
        resampling = (
            conf["resampling"]["_target_"].split(".")[-1]
            if "resampling" in conf
            else "None"
        )
        resampling = resampling if resampling != "NoUnderSampler" else "None"
        # Collect results in a list of dicts
        if remove_xgboost and model_name == "XGBClassifier":
            continue
        results.append(
            {
                "Dataset": dataset,
                "Side": side,
                "Model": model_name,
                "Resampling": resampling,
                "Features": features_name,
                "Aggregator": aggregator,
                "Validation": validation_method,
                "Detailed Report": report,
                **report_results,
            }
        )

    # After the loop, display as a table
    df_results = pd.DataFrame(results)
    for (dataset, side, resampling), group in df_results.groupby(
        ["Dataset", "Side", "Resampling"]
    ):
        display(
            HTML(
                f"""
            <div style='background-color:#ffe6e6; padding:18px; margin:10px 0; border-radius:8px;'>
                <h2 style='color:#b30000; margin:0; font-size:2em;'>
                Results for Dataset: <i>{dataset}</i>, Side: <i>{side}</i>, resampling: <i>{resampling}</i>
                </h2>
            </div>
            """
            )
        )
        display(
            group.sort_values(by=["Model", "Features", "Aggregator"])
            .drop(columns=["Detailed Report"])
            .drop_duplicates()
        )
        if "aggregator" in test_args.keys() and group["Aggregator"].nunique() > 2:
            group = group[
                (group["Aggregator"] == test_args["aggregator"])
                | (group["Aggregator"].isnull())
            ]

        unravelled_detailed_results = {
            metric: {} for metric in group["Detailed Report"].iloc[0].columns
        }
        for i, model_results in group.iterrows():
            for metric in model_results["Detailed Report"].columns:
                cv_results = model_results["Detailed Report"][metric]
                unravelled_detailed_results[metric][
                    (model_results["Model"], model_results["Features"])
                ] = cv_results

        if which_test == "friedmann-nemenyi":
            friedmann_nemenyi_test(unravelled_detailed_results)
        elif which_test == "alignedfriedmann-holm":
            aligned_friedmann_holm_test(unravelled_detailed_results)
        else:
            raise ValueError(f"Unknown test: {which_test}")

    return results

In [5]:
results_path: str = "../outputs/"
all_results = list(Path(results_path).glob("*/*/*/reports.csv"))
results_path_adula: str = "../outputs_adula/"
all_results += list(Path(results_path_adula).glob("*/*/*/reports.csv"))


results_lopo = present_results(
    all_results,
    val_method="lopo",
    remove_xgboost=True,
    remove_chronos_small_from_test=True,
    which_test="friedmann-nemenyi",
    test_args={
        "aggregator": "MeanChanAggregator"}
)

,Dataset,Side,Model,Resampling,Features,Aggregator,Validation,accuracy_score avg,accuracy_score sem,balanced_accuracy_score avg,balanced_accuracy_score sem,matthews_corrcoef avg,matthews_corrcoef sem,f1_score avg,f1_score sem,precision_score avg,precision_score sem,recall_score avg,recall_score sem,roc_auc_score avg,roc_auc_score sem
6,apsync,engagement_10sec,DummyClassifier,None,HandcraftedFeatureExtractor,None,LOPO,0.421389,0.092253,0.518713,0.027086,0.034524,0.051993,0.318314,0.174510,0.272775,0.169406,0.457165,0.268180,0.518713,0.027086
8,apsync,engagement_10sec,LogisticRegression,None,AutonLab/MOMENT-1-large,MeanChanAggregator,LOPO,0.484316,0.153246,0.500000,0.000000,0.000000,0.000000,0.065514,0.129718,0.042503,0.084156,0.142857,0.282857,0.500000,0.000000
5,apsync,engagement_10sec,LogisticRegression,None,HandcraftedFeatureExtractor,None,LOPO,0.627536,0.107493,0.641607,0.092030,0.282201,0.157940,0.535047,0.163484,0.623977,0.168456,0.580068,0.249043,0.641607,0.092030
4,apsync,engagement_10sec,LogisticRegression,None,amazon/chronos-t5-large,MeanChanAggregator,LOPO,0.482812,0.153343,0.498413,0.003143,-0.015532,0.030753,0.065514,0.129718,0.042503,0.084156,0.142857,0.282857,0.498413,0.003143
7,apsync,engagement_10sec,LogisticRegression,None,amazon/chronos-t5-small,MeanChanAggregator,LOPO,0.499654,0.086121,0.500339,0.050637,-0.003384,0.100121,0.297661,0.185441,0.311768,0.213402,0.328211,0.214678,0.500339,0.050637
3,apsync,engagement_10sec,LogisticRegression,None,ibm-granite/granite-timeseries-patchtsmixer,MeanChanAggregator,LOPO,0.425518,0.088017,0.492814,0.032105,-0.009583,0.084659,0.288127,0.098984,0.466334,0.196568,0.360991,0.234210,0.492814,0.032105
9,apsync,engagement_10sec,LogisticRegression,None,paris-noah/Mantis-8M,None,LOPO,0.560433,0.079193,0.564281,0.072435,0.140943,0.140576,0.507337,0.157066,0.529815,0.212513,0.573615,0.174038,0.564281,0.072435


Friedman Test accuracy_score: statistic=10.146, pvalue=0.119
Nemenyi post-hoc test


DummyClassifier  \
                                                               HandcraftedFeatureExtractor   
DummyClassifier    HandcraftedFeatureExtractor                                    1.000000   
LogisticRegression AutonLab/MOMENT-1-large                                        0.989917   
                   HandcraftedFeatureExtractor                                    0.220024   
                   amazon/chronos-t5-large                                        0.996250   
                   amazon/chronos-t5-small                                        0.676979   
                   ibm-granite/granite-timeseries-patchtsmixer                    1.000000   
                   paris-noah/Mantis-8M                                           0.676979   

                                                                    LogisticRegression  \
                                                               AutonLab/MOMENT-1-large   
DummyClassifier    HandcraftedFeatureExtractor                                0.989917   
LogisticRegression AutonLab/MOMENT-1-large                                    1.000000   
                   HandcraftedFeatureExtractor                                0.676979   
                   amazon/chronos-t5-large                                    1.000000   
                   amazon/chronos-t5-small                                    0.977501   
                   ibm-granite/granite-timeseries-patchtsmixer                0.989917   
                   paris-noah/Mantis-8M                                       0.977501   

                                                                                            \
                                                               HandcraftedFeatureExtractor   
DummyClassifier    HandcraftedFeatureExtractor                                    0.220024   
LogisticRegression AutonLab/MOMENT-1-large                                        0.676979   
                   HandcraftedFeatureExtractor                                    1.000000   
                   amazon/chronos-t5-large                                        0.594339   
                   amazon/chronos-t5-small                                        0.989917   
                   ibm-granite/granite-timeseries-patchtsmixer                    0.220024   
                   paris-noah/Mantis-8M                                           0.989917   

                                                                                        \
                                                               amazon/chronos-t5-large   
DummyClassifier    HandcraftedFeatureExtractor                                0.996250   
LogisticRegression AutonLab/MOMENT-1-large                                    1.000000   
                   HandcraftedFeatureExtractor                                0.594339   
                   amazon/chronos-t5-large                                    1.000000   
                   amazon/chronos-t5-small                                    0.956377   
                   ibm-granite/granite-timeseries-patchtsmixer                0.996250   
                   paris-noah/Mantis-8M                                       0.956377   

                                                                                        \
                                                               amazon/chronos-t5-small   
DummyClassifier    HandcraftedFeatureExtractor                                0.676979   
LogisticRegression AutonLab/MOMENT-1-large                                    0.977501   
                   HandcraftedFeatureExtractor                                0.989917   
                   amazon/chronos-t5-large                                    0.956377   
                   amazon/chronos-t5-small                                    1.000000   
                   ibm-granite/granite-timeseries-patchtsmixer                0.676979   
                   paris-noah/Mantis-8M

Friedman Test balanced_accuracy_score: statistic=12.946, pvalue=0.044
Nemenyi post-hoc test


DummyClassifier  \
                                                               HandcraftedFeatureExtractor   
DummyClassifier    HandcraftedFeatureExtractor                                    1.000000   
LogisticRegression AutonLab/MOMENT-1-large                                        0.998928   
                   HandcraftedFeatureExtractor                                    0.249290   
                   amazon/chronos-t5-large                                        0.996250   
                   amazon/chronos-t5-small                                        1.000000   
                   ibm-granite/granite-timeseries-patchtsmixer                    1.000000   
                   paris-noah/Mantis-8M                                           0.968186   

                                                                    LogisticRegression  \
                                                               AutonLab/MOMENT-1-large   
DummyClassifier    HandcraftedFeatureExtractor                                0.998928   
LogisticRegression AutonLab/MOMENT-1-large                                    1.000000   
                   HandcraftedFeatureExtractor                                0.078926   
                   amazon/chronos-t5-large                                    1.000000   
                   amazon/chronos-t5-small                                    0.996250   
                   ibm-granite/granite-timeseries-patchtsmixer                0.999501   
                   paris-noah/Mantis-8M                                       0.789690   

                                                                                            \
                                                               HandcraftedFeatureExtractor   
DummyClassifier    HandcraftedFeatureExtractor                                    0.249290   
LogisticRegression AutonLab/MOMENT-1-large                                        0.078926   
                   HandcraftedFeatureExtractor                                    1.000000   
                   amazon/chronos-t5-large                                        0.056196   
                   amazon/chronos-t5-small                                        0.314698   
                   ibm-granite/granite-timeseries-patchtsmixer                    0.220024   
                   paris-noah/Mantis-8M                                           0.822613   

                                                                                        \
                                                               amazon/chronos-t5-large   
DummyClassifier    HandcraftedFeatureExtractor                                0.996250   
LogisticRegression AutonLab/MOMENT-1-large                                    1.000000   
                   HandcraftedFeatureExtractor                                0.056196   
                   amazon/chronos-t5-large                                    1.000000   
                   amazon/chronos-t5-small                                    0.989917   
                   ibm-granite/granite-timeseries-patchtsmixer                0.997914   
                   paris-noah/Mantis-8M                                       0.716494   

                                                                                        \
                                                               amazon/chronos-t5-small   
DummyClassifier    HandcraftedFeatureExtractor                                1.000000   
LogisticRegression AutonLab/MOMENT-1-large                                    0.996250   
                   HandcraftedFeatureExtractor                                0.314698   
                   amazon/chronos-t5-large                                    0.989917   
                   amazon/chronos-t5-small                                    1.000000   
                   ibm-granite/granite-timeseries-patchtsmixer                0.999997   
                   paris-noah/Mantis-8M

...

DummyClassifier  \
                                                               HandcraftedFeatureExtractor   
DummyClassifier    HandcraftedFeatureExtractor                                    1.000000   
LogisticRegression AutonLab/MOMENT-1-large                                        0.093463   
                   HandcraftedFeatureExtractor                                    0.000150   
                   amazon/chronos-t5-large                                        0.000074   
                   amazon/chronos-t5-small                                        0.000259   
                   ibm-granite/granite-timeseries-patchtsmixer                    0.000064   
                   paris-noah/Mantis-8M                                           0.001076   

                                                                    LogisticRegression  \
                                                               AutonLab/MOMENT-1-large   
DummyClassifier    HandcraftedFeatureExtractor                                0.093463   
LogisticRegression AutonLab/MOMENT-1-large                                    1.000000   
                   HandcraftedFeatureExtractor                                0.573285   
                   amazon/chronos-t5-large                                    0.471953   
                   amazon/chronos-t5-small                                    0.653919   
                   ibm-granite/granite-timeseries-patchtsmixer                0.452122   
                   paris-noah/Mantis-8M                                       0.845331   

                                                                                            \
                                                               HandcraftedFeatureExtractor   
DummyClassifier    HandcraftedFeatureExtractor                                    0.000150   
LogisticRegression AutonLab/MOMENT-1-large                                        0.573285   
                   HandcraftedFeatureExtractor                                    1.000000   
                   amazon/chronos-t5-large                                        0.999999   
                   amazon/chronos-t5-small                                        1.000000   
                   ibm-granite/granite-timeseries-patchtsmixer                    0.999997   
                   paris-noah/Mantis-8M                                           0.999391   

                                                                                        \
                                                               amazon/chronos-t5-large   
DummyClassifier    HandcraftedFeatureExtractor                                0.000074   
LogisticRegression AutonLab/MOMENT-1-large                                    0.471953   
                   HandcraftedFeatureExtractor                                0.999999   
                   amazon/chronos-t5-large                                    1.000000   
                   amazon/chronos-t5-small                                    0.999969   
                   ibm-granite/granite-timeseries-patchtsmixer                1.000000   
                   paris-noah/Mantis-8M                                       0.996902   

                                                                                        \
                                                               amazon/chronos-t5-small   
DummyClassifier    HandcraftedFeatureExtractor                                0.000259   
LogisticRegression AutonLab/MOMENT-1-large                                    0.653919   
                   HandcraftedFeatureExtractor                                1.000000   
                   amazon/chronos-t5-large                                    0.999969   
                   amazon/chronos-t5-small                                    1.000000   
                   ibm-granite/granite-timeseries-patchtsmixer                0.999943   
                   paris-noah/Mantis-8M

Friedman Test recall_score: statistic=56.845, pvalue=0.000
Nemenyi post-hoc test


DummyClassifier  \
                                                               HandcraftedFeatureExtractor   
DummyClassifier    HandcraftedFeatureExtractor                                    1.000000   
LogisticRegression AutonLab/MOMENT-1-large                                        0.957155   
                   HandcraftedFeatureExtractor                                    0.000031   
                   amazon/chronos-t5-large                                        0.145149   
                   amazon/chronos-t5-small                                        0.177888   
                   ibm-granite/granite-timeseries-patchtsmixer                    0.001555   
                   paris-noah/Mantis-8M                                           0.052952   

                                                                    LogisticRegression  \
                                                               AutonLab/MOMENT-1-large   
DummyClassifier    HandcraftedFeatureExtractor                            9.571545e-01   
LogisticRegression AutonLab/MOMENT-1-large                                1.000000e+00   
                   HandcraftedFeatureExtractor                            1.414725e-07   
                   amazon/chronos-t5-large                                7.701812e-03   
                   amazon/chronos-t5-small                                1.057172e-02   
                   ibm-granite/granite-timeseries-patchtsmixer            1.697175e-05   
                   paris-noah/Mantis-8M                                   1.754997e-03   

                                                                                            \
                                                               HandcraftedFeatureExtractor   
DummyClassifier    HandcraftedFeatureExtractor                                3.096421e-05   
LogisticRegression AutonLab/MOMENT-1-large                                    1.414725e-07   
                   HandcraftedFeatureExtractor                                1.000000e+00   
                   amazon/chronos-t5-large                                    2.581681e-01   
                   amazon/chronos-t5-small                                    2.155438e-01   
                   ibm-granite/granite-timeseries-patchtsmixer                9.774333e-01   
                   paris-noah/Mantis-8M                                       4.919929e-01   

                                                                                        \
                                                               amazon/chronos-t5-large   
DummyClassifier    HandcraftedFeatureExtractor                                0.145149   
LogisticRegression AutonLab/MOMENT-1-large                                    0.007702   
                   HandcraftedFeatureExtractor                                0.258168   
                   amazon/chronos-t5-large                                    1.000000   
                   amazon/chronos-t5-small                                    1.000000   
                   ibm-granite/granite-timeseries-patchtsmixer                0.799745   
                   paris-noah/Mantis-8M                                       0.999734   

                                                                                        \
                                                               amazon/chronos-t5-small   
DummyClassifier    HandcraftedFeatureExtractor                                0.177888   
LogisticRegression AutonLab/MOMENT-1-large                                    0.010572   
                   HandcraftedFeatureExtractor                                0.215544   
                   amazon/chronos-t5-large                                    1.000000   
                   amazon/chronos-t5-small                                    1.000000   
                   ibm-granite/granite-timeseries-patchtsmixer                0.748591   
                   paris-noah/Mantis-8M

Friedman Test roc_auc_score: statistic=31.970, pvalue=0.000
Nemenyi post-hoc test


DummyClassifier  \
                                                               HandcraftedFeatureExtractor   
DummyClassifier    HandcraftedFeatureExtractor                                    1.000000   
LogisticRegression AutonLab/MOMENT-1-large                                        0.859153   
                   HandcraftedFeatureExtractor                                    0.000572   
                   amazon/chronos-t5-large                                        0.010572   
                   amazon/chronos-t5-small                                        0.001978   
                   ibm-granite/granite-timeseries-patchtsmixer                    0.001377   
                   paris-noah/Mantis-8M                                           0.015890   

                                                                    LogisticRegression  \
                                                               AutonLab/MOMENT-1-large   
DummyClassifier    HandcraftedFeatureExtractor                                0.859153   
LogisticRegression AutonLab/MOMENT-1-large                                    1.000000   
                   HandcraftedFeatureExtractor                                0.057631   
                   amazon/chronos-t5-large                                    0.322449   
                   amazon/chronos-t5-small                                    0.125956   
                   ibm-granite/granite-timeseries-patchtsmixer                0.100885   
                   paris-noah/Mantis-8M                                       0.394299   

                                                                                            \
                                                               HandcraftedFeatureExtractor   
DummyClassifier    HandcraftedFeatureExtractor                                    0.000572   
LogisticRegression AutonLab/MOMENT-1-large                                        0.057631   
                   HandcraftedFeatureExtractor                                    1.000000   
                   amazon/chronos-t5-large                                        0.989572   
                   amazon/chronos-t5-small                                        0.999943   
                   ibm-granite/granite-timeseries-patchtsmixer                    0.999993   
                   paris-noah/Mantis-8M                                           0.977433   

                                                                                        \
                                                               amazon/chronos-t5-large   
DummyClassifier    HandcraftedFeatureExtractor                                0.010572   
LogisticRegression AutonLab/MOMENT-1-large                                    0.322449   
                   HandcraftedFeatureExtractor                                0.989572   
                   amazon/chronos-t5-large                                    1.000000   
                   amazon/chronos-t5-small                                    0.999391   
                   ibm-granite/granite-timeseries-patchtsmixer                0.998282   
                   paris-noah/Mantis-8M                                       1.000000   

                                                                                        \
                                                               amazon/chronos-t5-small   
DummyClassifier    HandcraftedFeatureExtractor                                0.001978   
LogisticRegression AutonLab/MOMENT-1-large                                    0.125956   
                   HandcraftedFeatureExtractor                                0.999943   
                   amazon/chronos-t5-large                                    0.999391   
                   amazon/chronos-t5-small                                    1.000000   
                   ibm-granite/granite-timeseries-patchtsmixer                1.000000   
                   paris-noah/Mantis-8M

In [3]:
results_lopo = present_results(
    all_results,
    val_method="tacv",
    remove_xgboost=True,
    remove_chronos_small_from_test=True,
)

,Dataset,Side,Model,Resampling,Features,Aggregator,Validation,accuracy_score avg,accuracy_score sem,balanced_accuracy_score avg,balanced_accuracy_score sem,matthews_corrcoef avg,matthews_corrcoef sem,f1_score avg,f1_score sem,precision_score avg,precision_score sem,recall_score avg,recall_score sem,roc_auc_score avg,roc_auc_score sem
4,apsync,engagement_10sec,DummyClassifier,None,HandcraftedFeatureExtractor,None,TACV,0.478237,0.051894,0.500512,0.031328,0.001778,0.061588,0.387970,0.202726,0.357265,0.200925,0.443405,0.221716,0.500512,0.031328
6,apsync,engagement_10sec,LogisticRegression,None,AutonLab/MOMENT-1-large,MeanChanAggregator,TACV,0.534116,0.090357,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000
3,apsync,engagement_10sec,LogisticRegression,None,HandcraftedFeatureExtractor,None,TACV,0.640500,0.131275,0.615714,0.136109,0.241794,0.286743,0.462302,0.302731,0.625714,0.362842,0.466578,0.334841,0.615714,0.136109
2,apsync,engagement_10sec,LogisticRegression,None,amazon/chronos-t5-large,MeanChanAggregator,TACV,0.534116,0.090357,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000
5,apsync,engagement_10sec,LogisticRegression,None,amazon/chronos-t5-small,MeanChanAggregator,TACV,0.508916,0.058305,0.523843,0.049311,0.041462,0.110617,0.297648,0.210281,0.364110,0.201832,0.312121,0.266114,0.523843,0.049311
1,apsync,engagement_10sec,LogisticRegression,None,ibm-granite/granite-timeseries-patchtsmixer,MeanChanAggregator,TACV,0.504563,0.053762,0.519786,0.050948,0.021321,0.133771,0.361768,0.191620,0.405507,0.242627,0.368939,0.210530,0.519786,0.050948


Friedman Test accuracy_score: statistic=2.898, pvalue=0.716
Nemenyi post-hoc test


DummyClassifier  \
                                                               HandcraftedFeatureExtractor   
DummyClassifier    HandcraftedFeatureExtractor                                    1.000000   
LogisticRegression AutonLab/MOMENT-1-large                                        0.973980   
                   HandcraftedFeatureExtractor                                    0.594680   
                   amazon/chronos-t5-large                                        0.973980   
                   amazon/chronos-t5-small                                        0.984591   
                   ibm-granite/granite-timeseries-patchtsmixer                    0.973980   

                                                                    LogisticRegression  \
                                                               AutonLab/MOMENT-1-large   
DummyClassifier    HandcraftedFeatureExtractor                                0.973980   
LogisticRegression AutonLab/MOMENT-1-large                                    1.000000   
                   HandcraftedFeatureExtractor                                0.958997   
                   amazon/chronos-t5-large                                    1.000000   
                   amazon/chronos-t5-small                                    0.999999   
                   ibm-granite/granite-timeseries-patchtsmixer                1.000000   

                                                                                            \
                                                               HandcraftedFeatureExtractor   
DummyClassifier    HandcraftedFeatureExtractor                                    0.594680   
LogisticRegression AutonLab/MOMENT-1-large                                        0.958997   
                   HandcraftedFeatureExtractor                                    1.000000   
                   amazon/chronos-t5-large                                        0.958997   
                   amazon/chronos-t5-small                                        0.938967   
                   ibm-granite/granite-timeseries-patchtsmixer                    0.958997   

                                                                                        \
                                                               amazon/chronos-t5-large   
DummyClassifier    HandcraftedFeatureExtractor                                0.973980   
LogisticRegression AutonLab/MOMENT-1-large                                    1.000000   
                   HandcraftedFeatureExtractor                                0.958997   
                   amazon/chronos-t5-large                                    1.000000   
                   amazon/chronos-t5-small                                    0.999999   
                   ibm-granite/granite-timeseries-patchtsmixer                1.000000   

                                                                                        \
                                                               amazon/chronos-t5-small   
DummyClassifier    HandcraftedFeatureExtractor                                0.984591   
LogisticRegression AutonLab/MOMENT-1-large                                    0.999999   
                   HandcraftedFeatureExtractor                                0.938967   
                   amazon/chronos-t5-large                                    0.999999   
                   amazon/chronos-t5-small                                    1.000000   
                   ibm-granite/granite-timeseries-patchtsmixer                0.999999   

                                                                                                            
                                                               ibm-granite/granite-timeseries-patchtsmixer  
DummyClassifier    HandcraftedFeatureExtractor                                                    0.973980  
LogisticRegression AutonLab/MOMENT-1-large                                     

Friedman Test balanced_accuracy_score: statistic=2.500, pvalue=0.776
Nemenyi post-hoc test


DummyClassifier  \
                                                               HandcraftedFeatureExtractor   
DummyClassifier    HandcraftedFeatureExtractor                                    1.000000   
LogisticRegression AutonLab/MOMENT-1-large                                        0.999999   
                   HandcraftedFeatureExtractor                                    0.845079   
                   amazon/chronos-t5-large                                        0.999999   
                   amazon/chronos-t5-small                                        0.958997   
                   ibm-granite/granite-timeseries-patchtsmixer                    0.999420   

                                                                    LogisticRegression  \
                                                               AutonLab/MOMENT-1-large   
DummyClassifier    HandcraftedFeatureExtractor                                0.999999   
LogisticRegression AutonLab/MOMENT-1-large                                    1.000000   
                   HandcraftedFeatureExtractor                                0.882097   
                   amazon/chronos-t5-large                                    1.000000   
                   amazon/chronos-t5-small                                    0.973980   
                   ibm-granite/granite-timeseries-patchtsmixer                0.999858   

                                                                                            \
                                                               HandcraftedFeatureExtractor   
DummyClassifier    HandcraftedFeatureExtractor                                    0.845079   
LogisticRegression AutonLab/MOMENT-1-large                                        0.882097   
                   HandcraftedFeatureExtractor                                    1.000000   
                   amazon/chronos-t5-large                                        0.882097   
                   amazon/chronos-t5-small                                        0.999420   
                   ibm-granite/granite-timeseries-patchtsmixer                    0.958997   

                                                                                        \
                                                               amazon/chronos-t5-large   
DummyClassifier    HandcraftedFeatureExtractor                                0.999999   
LogisticRegression AutonLab/MOMENT-1-large                                    1.000000   
                   HandcraftedFeatureExtractor                                0.882097   
                   amazon/chronos-t5-large                                    1.000000   
                   amazon/chronos-t5-small                                    0.973980   
                   ibm-granite/granite-timeseries-patchtsmixer                0.999858   

                                                                                        \
                                                               amazon/chronos-t5-small   
DummyClassifier    HandcraftedFeatureExtractor                                0.958997   
LogisticRegression AutonLab/MOMENT-1-large                                    0.973980   
                   HandcraftedFeatureExtractor                                0.999420   
                   amazon/chronos-t5-large                                    0.973980   
                   amazon/chronos-t5-small                                    1.000000   
                   ibm-granite/granite-timeseries-patchtsmixer                0.995925   

                                                                                                            
                                                               ibm-granite/granite-timeseries-patchtsmixer  
DummyClassifier    HandcraftedFeatureExtractor                                                    0.999420  
LogisticRegression AutonLab/MOMENT-1-large                                     

...

DummyClassifier  \
                                                               HandcraftedFeatureExtractor   
DummyClassifier    HandcraftedFeatureExtractor                                    1.000000   
LogisticRegression AutonLab/MOMENT-1-large                                        0.928962   
                   HandcraftedFeatureExtractor                                    0.478211   
                   amazon/chronos-t5-large                                        0.013423   
                   amazon/chronos-t5-small                                        0.008059   
                   ibm-granite/granite-timeseries-patchtsmixer                    0.339140   
                   paris-noah/Mantis-8M                                           0.223897   

                                                                    LogisticRegression  \
                                                               AutonLab/MOMENT-1-large   
DummyClassifier    HandcraftedFeatureExtractor                                0.928962   
LogisticRegression AutonLab/MOMENT-1-large                                    1.000000   
                   HandcraftedFeatureExtractor                                0.984540   
                   amazon/chronos-t5-large                                    0.259158   
                   amazon/chronos-t5-small                                    0.191943   
                   ibm-granite/granite-timeseries-patchtsmixer                0.948498   
                   paris-noah/Mantis-8M                                       0.876775   

                                                                                            \
                                                               HandcraftedFeatureExtractor   
DummyClassifier    HandcraftedFeatureExtractor                                    0.478211   
LogisticRegression AutonLab/MOMENT-1-large                                        0.984540   
                   HandcraftedFeatureExtractor                                    1.000000   
                   amazon/chronos-t5-large                                        0.766379   
                   amazon/chronos-t5-small                                        0.675740   
                   ibm-granite/granite-timeseries-patchtsmixer                    0.999991   
                   paris-noah/Mantis-8M                                           0.999459   

                                                                                        \
                                                               amazon/chronos-t5-large   
DummyClassifier    HandcraftedFeatureExtractor                                0.013423   
LogisticRegression AutonLab/MOMENT-1-large                                    0.259158   
                   HandcraftedFeatureExtractor                                0.766379   
                   amazon/chronos-t5-large                                    1.000000   
                   amazon/chronos-t5-small                                    0.999999   
                   ibm-granite/granite-timeseries-patchtsmixer                0.876775   
                   paris-noah/Mantis-8M                                       0.948498   

                                                                                        \
                                                               amazon/chronos-t5-small   
DummyClassifier    HandcraftedFeatureExtractor                                0.008059   
LogisticRegression AutonLab/MOMENT-1-large                                    0.191943   
                   HandcraftedFeatureExtractor                                0.675740   
                   amazon/chronos-t5-large                                    0.999999   
                   amazon/chronos-t5-small                                    1.000000   
                   ibm-granite/granite-timeseries-patchtsmixer                0.807113   
                   paris-noah/Mantis-8M

Friedman Test recall_score: statistic=22.390, pvalue=0.001
Nemenyi post-hoc test


DummyClassifier  \
                                                               HandcraftedFeatureExtractor   
DummyClassifier    HandcraftedFeatureExtractor                                    1.000000   
LogisticRegression AutonLab/MOMENT-1-large                                        0.999999   
                   HandcraftedFeatureExtractor                                    0.021791   
                   amazon/chronos-t5-large                                        0.675740   
                   amazon/chronos-t5-small                                        0.297637   
                   ibm-granite/granite-timeseries-patchtsmixer                    0.096148   
                   paris-noah/Mantis-8M                                           0.096148   

                                                                    LogisticRegression  \
                                                               AutonLab/MOMENT-1-large   
DummyClassifier    HandcraftedFeatureExtractor                                0.999999   
LogisticRegression AutonLab/MOMENT-1-large                                    1.000000   
                   HandcraftedFeatureExtractor                                0.013423   
                   amazon/chronos-t5-large                                    0.577610   
                   amazon/chronos-t5-small                                    0.223897   
                   ibm-granite/granite-timeseries-patchtsmixer                0.065133   
                   paris-noah/Mantis-8M                                       0.065133   

                                                                                            \
                                                               HandcraftedFeatureExtractor   
DummyClassifier    HandcraftedFeatureExtractor                                    0.021791   
LogisticRegression AutonLab/MOMENT-1-large                                        0.013423   
                   HandcraftedFeatureExtractor                                    1.000000   
                   amazon/chronos-t5-large                                        0.675740   
                   amazon/chronos-t5-small                                        0.948498   
                   ibm-granite/granite-timeseries-patchtsmixer                    0.998694   
                   paris-noah/Mantis-8M                                           0.998694   

                                                                                        \
                                                               amazon/chronos-t5-large   
DummyClassifier    HandcraftedFeatureExtractor                                0.675740   
LogisticRegression AutonLab/MOMENT-1-large                                    0.577610   
                   HandcraftedFeatureExtractor                                0.675740   
                   amazon/chronos-t5-large                                    1.000000   
                   amazon/chronos-t5-small                                    0.997234   
                   ibm-granite/granite-timeseries-patchtsmixer                0.928962   
                   paris-noah/Mantis-8M                                       0.928962   

                                                                                        \
                                                               amazon/chronos-t5-small   
DummyClassifier    HandcraftedFeatureExtractor                                0.297637   
LogisticRegression AutonLab/MOMENT-1-large                                    0.223897   
                   HandcraftedFeatureExtractor                                0.948498   
                   amazon/chronos-t5-large                                    0.997234   
                   amazon/chronos-t5-small                                    1.000000   
                   ibm-granite/granite-timeseries-patchtsmixer                0.998694   
                   paris-noah/Mantis-8M

Friedman Test roc_auc_score: statistic=19.527, pvalue=0.003
Nemenyi post-hoc test


DummyClassifier  \
                                                               HandcraftedFeatureExtractor   
DummyClassifier    HandcraftedFeatureExtractor                                    1.000000   
LogisticRegression AutonLab/MOMENT-1-large                                        0.999459   
                   HandcraftedFeatureExtractor                                    0.079418   
                   amazon/chronos-t5-large                                        0.065133   
                   amazon/chronos-t5-small                                        0.053044   
                   ibm-granite/granite-timeseries-patchtsmixer                    0.163298   
                   paris-noah/Mantis-8M                                           0.065133   

                                                                    LogisticRegression  \
                                                               AutonLab/MOMENT-1-large   
DummyClassifier    HandcraftedFeatureExtractor                                0.999459   
LogisticRegression AutonLab/MOMENT-1-large                                    1.000000   
                   HandcraftedFeatureExtractor                                0.223897   
                   amazon/chronos-t5-large                                    0.191943   
                   amazon/chronos-t5-small                                    0.163298   
                   ibm-granite/granite-timeseries-patchtsmixer                0.383365   
                   paris-noah/Mantis-8M                                       0.191943   

                                                                                            \
                                                               HandcraftedFeatureExtractor   
DummyClassifier    HandcraftedFeatureExtractor                                    0.079418   
LogisticRegression AutonLab/MOMENT-1-large                                        0.223897   
                   HandcraftedFeatureExtractor                                    1.000000   
                   amazon/chronos-t5-large                                        1.000000   
                   amazon/chronos-t5-small                                        0.999999   
                   ibm-granite/granite-timeseries-patchtsmixer                    0.999949   
                   paris-noah/Mantis-8M                                           1.000000   

                                                                                        \
                                                               amazon/chronos-t5-large   
DummyClassifier    HandcraftedFeatureExtractor                                0.065133   
LogisticRegression AutonLab/MOMENT-1-large                                    0.191943   
                   HandcraftedFeatureExtractor                                1.000000   
                   amazon/chronos-t5-large                                    1.000000   
                   amazon/chronos-t5-small                                    1.000000   
                   ibm-granite/granite-timeseries-patchtsmixer                0.999812   
                   paris-noah/Mantis-8M                                       1.000000   

                                                                                        \
                                                               amazon/chronos-t5-small   
DummyClassifier    HandcraftedFeatureExtractor                                0.053044   
LogisticRegression AutonLab/MOMENT-1-large                                    0.163298   
                   HandcraftedFeatureExtractor                                0.999999   
                   amazon/chronos-t5-large                                    1.000000   
                   amazon/chronos-t5-small                                    1.000000   
                   ibm-granite/granite-timeseries-patchtsmixer                0.999459   
                   paris-noah/Mantis-8M